Text Generation from Shakespeare Play using Markov Chains
============================================

In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
import re

### Reading a Text File

In [2]:
filepath = "./data_shakespeare.txt"

with open(filepath, 'r') as f:
    shakespeare_txt = f.read()

### Data Preprocessing

In [3]:
shakespeare_txt = re.sub("([.,!?:;])", r" \1 ", shakespeare_txt)

#### Using Tokenizer class of keras library

In [4]:
tokenizer = keras.preprocessing.text.Tokenizer(filters = "\"#$%&()*+-/<=>@[\\]^_`{|}~\t\n", char_level = False)
tokenizer.fit_on_texts([shakespeare_txt])

#### number of distinct words

In [5]:
max_id = len(tokenizer.word_index)
max_id

12638

#### Total Words in the file

In [6]:
word_counts = dict(tokenizer.word_counts)
total_words = sum(word_counts.values())
total_words

250398

### Create Embeddings/Vectors for Tokens

In [7]:
[embeddings] = np.array(tokenizer.texts_to_sequences([shakespeare_txt])) - 1

In [8]:
embeddings.shape

(250398,)

In [9]:
embeddings[0]

93

# Understanding Markov Chains

#### Markov chains are random determined processes with a finite set of states that move from one state to another. These sets of transitions from state to state are determined by some probability distribution. 

Text as a Markov Chain
----------------------

**The cat ran over the dog.**

![alt text](images/markov_text1.png "Text")


A Markov chain typically consists of two entities: A transition matrix and an initial state vector. 

As we saw above, the next state in the chain depends on the probability distribution of the previous state. These probabilities are represented in the form of a transition matrix.

Transition Matrix representation __rows are current state, columns are next state__:

| | the | cat | ran | over | dog | . |
| --- | --- | --- | --- | --- | --- | --- |
| **the** | 0 | 0.5 | 0 | 0 | 0.5 | 0 |
| **cat** | 0 | 0 | 1 | 0 | 0 | 0 |
| **ran** | 0 | 0 | 0 | 1 | 0 | 0 |
| **over** | 1 | 0 | 0 | 0 | 0 | 0 |
| **dog** | 0 | 0 | 0 | 0 | 0 | 1 |
| **.** | 0 | 0 | 0 | 0 | 0 | 1 |

To make the implementation of Markov chains easy, you can make use of the built-in package known as markovify. 

To install this use the following command 

`pip install markovify`

However, for this project we shall be building the __matrices from scratch__ to better understand the implementation

### Building transition matrix
- Most of the elements would be zero. 
- Hence using a sparse array representation
- csr - Compressed Sparse Row matrix or lil - List of Lists Matrix
- We would use lil_matrix. It is more efficient

Creating few helper variables to assist in building transition matrix

In [10]:
### Can be created by code or we can use the tokenizer object here to create the same
words_idxs = {key:value - 1 for key, value in tokenizer.word_index.items()}
idxs_words = {key-1:value for key, value in tokenizer.index_word.items()}

In [11]:
distinct_states = list(words_idxs.keys())

In [12]:
from scipy.sparse import lil_matrix
tr_mat = lil_matrix(
    (len(distinct_states), len(distinct_states)),
    dtype = float
)

To build the transition matrix we will loop over all tokens/embeddings and increment their respective place in the transition matrix (as above in the example).

We call the rows of a transition matrix as current states and columns as next states. We will increment the cell by 1 whenever we observe a transition from one particular current state to next state.  To understand better look at the table above.

In [13]:
%%time
### Iterate upto the second last token - Since that token has no next transition
for i in range(len(embeddings) - 1):
    row = embeddings[i]
    col = embeddings[i + 1]
    tr_mat[row, col] += 1

Wall time: 1.75 s


### Text Generation Algorithm

Now that we have the transition matrix. Let's generate text by following the below pseudo code:
- Get a random integer between (0, distinct_states). This will be the index of the starting state/word
- Get the word for this index
- Loop over to generate a fixed number of sentences or it would go on generating text endlessly:
    - Get the current state row of the transition matrix. It shows all the counts that is possible from current state to its next state in the columns
    - Convert the counts to probabilities by dividing by the row's sum
    - Choose a random choice from these probabilities using numpy's random choice
    - Get the next state word from the dictionary we have
    - if the next state is a sentence terminator increment num_sentences
    - else add the next_state's word to the output
    - update the start state to be the next state to get the next states and so on....
    
    
- We have also used capitalize() method just to capitalize the starting of each sentence

In [14]:
import numpy as np

start_state_index = np.random.randint(len(distinct_states)) # Random Starting Idx
start_state = idxs_words[start_state_index] # Random Starting Word
num_sentences = 0 # To generate fixed num of sentences
output = start_state.capitalize() # Initializing the output variable with start word
capitalize = False

while num_sentences < 5: # Generating 3 sentences
#     print(start_state)
    row = tr_mat[words_idxs[start_state], :] # Getting the row of possible transitions
    probabilities = row / row.sum() # Converting into probabilities
    probabilities = probabilities.toarray()[0] # numpy array conversion
    
    ## Chhosing the next state index using numpy random choice
    next_state_index = np.random.choice(
        len(distinct_states),
        1, 
        p = probabilities
    )
    
    # get next state using the index
    next_state = idxs_words[next_state_index[0]]
    
    if next_state in ('.', '!', '?'): # looking for sentence terminator
        num_sentences += 1
        output += next_state + "\n\n"
        capitalize = True
    elif next_state == ",":
        output += next_state
    else:
        if capitalize:
            output += next_state.capitalize()
            capitalize = False
        else:
            output += " " + next_state
    start_state = next_state # Updating the start state with the next state
    
print(output)

Object cheer these eyes shall they whom thou shalt know, in his royal realm ; and will murder : and be my wooing, i spill'd ; go with kind one ; he not so much fills mine ear ; worthy a seeming!

To infliction, make prepare me free desire it not only, will effects bianca's grief.

King my manors, and bid false.

Did tell thee a kiss thy help our throne, rave, boast itself the blossoms of behavior, knock me looks my soul's peril to make the prince is cheered by their abominable : within the men hate of some sufficient to the duke vincentio : yonder is bad enough.

Say, is my dear account this order?




k-Word Markov Chain
-------------------

**The cat ran over the dog.**

![alt text](images/markov_text2.png "Text")

In a k word markov chain we have k words/tokens as one single state. Hence the transition matrix below. <br />
Matrix representation (rows are current state, columns are next state):

| | the cat | cat ran | ran over | over the | the dog | dog. |
| --- | --- | --- | --- | --- | --- | --- |
| **the cat**  | 0 | 1 | 0 | 0 | 0 | 0 |
| **cat ran**  | 0 | 0 | 1 | 0 | 0 | 0 |
| **ran over** | 0 | 0 | 0 | 1 | 0 | 0 |
| **over the** | 0 | 0 | 0 | 0 | 1 | 0 |
| **the dog**  | 0 | 0 | 0 | 0 | 0 | 1 |
| **dog.**     | 0 | 0 | 0 | 0 | 0 | 1 |



Creating k-word states for the transition matrix

In [15]:
k = 3
embedding_states = [tuple(embeddings[i : i + k]) for i in range(len(embeddings) - k + 1)]
word_states = [tuple([idxs_words[st] for st in state]) for state in embedding_states ]
distinct_states = list(set(word_states))

#### Total distinct states formed of k-words/tokens

In [16]:
len(distinct_states)

193953

In [17]:
from scipy.sparse import lil_matrix
tr_mat = lil_matrix(
    (len(distinct_states), len(distinct_states)),
    dtype = float
)

### Creating a dictionary of state, id_num to get to id_num quickly from state
state_index = dict(
    [(state, idx_num) for idx_num, state in enumerate(distinct_states)]
)


In [18]:
%%time
### Iterate upto the last "k - 1" tokens - Since that token has no next transition
for i in range(len(embeddings) - k):
    row = state_index[word_states[i]]
    col = state_index[word_states[i + 1]]
    tr_mat[row, col] += 1

Wall time: 1.77 s


In [19]:
import numpy as np

start_state_index = np.random.randint(len(distinct_states)) # Random Starting Idx
start_state = distinct_states[start_state_index] # Random Starting Word
num_sentences = 0 # To generate fixed num of sentences
output = ' '.join(start_state).capitalize() # Initializing the output variable with start word
capitalize = False

while num_sentences < 5: # Generating 3 sentences
    row = tr_mat[state_index[start_state], :] # Getting the row of possible transitions
    probabilities = row / row.sum() # Converting into probabilities
    probabilities = probabilities.toarray()[0] # numpy array conversion
    
    ## Chhosing the next state index using numpy random choice
    next_state_index = np.random.choice(
        len(distinct_states), 
        1,
        p = probabilities
    )
    
    # get next state using the index
    next_state = distinct_states[next_state_index[0]]
    
    if next_state[-1] in ('.', '!', '?'): # looking for sentence terminator
        num_sentences += 1
        output += next_state[-1] + "\n\n"
        capitalize = True
    elif next_state[-1] in (",", ":", ";"):
        output += next_state[-1]
    else:
        if capitalize:
            output += next_state[-1].capitalize()
            capitalize = False
        else:
            output += " " + next_state[-1]
    
    # print("y")
    start_state = next_state # Updating the start state with the next state
    
print(output)

Soul ! a' was a merry man took up the child: 'yea, ' quoth he; and swore so loud, that, being mad herself, she's madly mated.

Gremio: o, bid me devise some mean to rid her from this second marriage, or in my cell there would she kill herself.

Then gave i her, so tutor'd by my art, a sleeping potion; which so drew the rest of the eight.

Will you laugh me asleep, for i never saw a vessel of like sorrow, so fill'd and so becoming: in pure white robes, like very sanctity, she did approach my cabin where i lay; thrice bow'd before me, and think upon my bidding.

Antigonus: hang all the husbands that cannot do that feat, you'll leave yourself hardly one subject.


